In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings(action='ignore')
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.formula.api as formula
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels as sm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LinearRegression,Lasso,Ridge
#from sklearn.externals import joblib
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
df = pd.read_excel (r'C:\Users\Kira\Desktop\Internship\Data\Delhi_Data-20210529T043217Z-001\Delhi_Data\2013-2020-RK-PURAM-final-dataset\RK_PURAM_DATA_0null.xlsx')
#df = pd.read_excel (r'D:\Downloads\rk_month.xlsx')
df=df.set_index('From Date')

In [15]:
df.head()

,PM2.5,PM1,NO,NO2,NOx,NH3,SO2,CO,Ozone
From Date,,,,,,,,,
2013-01-01,462.25,571.68,274.61,15.00,474.39,108.30,23.00,121.00,4.50
2013-01-02,331.93,417.23,102.16,28.68,170.48,59.99,19.98,2.68,2.64
2013-01-03,220.60,537.10,75.22,37.13,144.90,65.49,20.55,1.66,15.93
2013-01-04,295.63,634.82,116.30,67.87,226.48,103.64,20.97,2.67,23.15
2013-01-05,245.00,536.55,51.00,52.24,119.99,101.44,20.26,1.78,24.65


many places in RK puram were in containment zones for unlock 1.0 and unlock 2.0. So we take the range to consider our drop fro, 22 march to 31 July

In [37]:
frames=[df.loc["2013-03-24":"2013-07-31"],df.loc["2014-03-24":"2014-07-31"],df.loc["2015-03-24":"2015-07-31"],df.loc["2016-03-24":"2016-07-31"],df.loc["2017-03-24":"2017-07-31"],df.loc["2018-03-24":"2018-07-31"],df.loc["2019-03-24":"2019-07-31"]]
noLock_data=pd.concat(frames)

In [58]:
print('pollutant median',noLock_data['PM2.5'].median())
print('pollutant mean',noLock_data['PM2.5'].mean())
print('pollutant min',noLock_data['PM2.5'].min())
print('pollutant max',noLock_data['PM2.5'].max())
print(noLock_data[['From Date','PM2.5']].loc[noLock_data['PM2.5']==noLock_data['PM2.5'].min()])
print(noLock_data[['From Date','PM2.5']].loc[noLock_data['PM2.5']==noLock_data['PM2.5'].max()])

pollutant median 76.06833333333333
pollutant mean 86.00729120879117
pollutant min 9.83
pollutant max 287.42
     From Date  PM2.5
479 2016-06-21   9.83
     From Date   PM2.5
454 2016-05-27  287.42


In [41]:
lock2020=df.loc["2020-03-24":"2020-07-31"]

In [43]:
lock2020.loc[lock2020['PM2.5']==lock2020['PM2.5'].min()]

,From Date,PM2.5,PM1,NO,NO2,NOx,NH3,SO2,CO,Ozone
120,2020-07-22,6.22,36.43,8.8,31.93,23.85,24.64,8.25,0.6,9.51


In [63]:
print('pollutant median',lock2020['PM2.5'].median())
print('pollutant mean',lock2020['PM2.5'].mean())
print('pollutant min',lock2020['PM2.5'].min())
print('pollutant max',lock2020['PM2.5'].max())
print(lock2020[['From Date','PM2.5']].loc[lock2020['PM2.5']==lock2020['PM2.5'].min()])
print(lock2020[['From Date','PM2.5']].loc[lock2020['PM2.5']==lock2020['PM2.5'].max()])

pollutant median 29.72
pollutant mean 32.99973076923078
pollutant min 6.22
pollutant max 105.0
     From Date  PM2.5
120 2020-07-22   6.22
    From Date  PM2.5
13 2020-04-06  105.0


In [75]:
framesaf=[df.loc["2013-08-01":"2013-12-31"],df.loc["2014-08-01":"2014-12-31"],df.loc["2015-08-01":"2015-12-31"],df.loc["2016-08-01":"2016-12-31"],df.loc["2017-08-01":"2017-12-31"],df.loc["2018-08-01":"2018-12-31"],df.loc["2019-08-01":"2019-12-31"]]
afterlockprev=pd.concat(framesaf)
afterlockprev=afterlockprev.reset_index()


In [76]:
print('pollutant median',afterlockprev['PM2.5'].median())
print('pollutant mean',afterlockprev['PM2.5'].mean())
print('pollutant min',afterlockprev['PM2.5'].min())
print('pollutant max',afterlockprev['PM2.5'].max())
print(afterlockprev[['From Date','PM2.5']].loc[afterlockprev['PM2.5']==afterlockprev['PM2.5'].min()])
print(afterlockprev[['From Date','PM2.5']].loc[afterlockprev['PM2.5']==afterlockprev['PM2.5'].max()])

pollutant median 102.98
pollutant mean 139.1472194211018
pollutant min -1.88
pollutant max 713.12
     From Date  PM2.5
604 2016-12-24  -1.88
     From Date   PM2.5
556 2016-11-06  713.12


In [80]:
afterlock2020=df.loc["2020-08-01":"2020-12-31"]
afterlock2020=afterlock2020.reset_index()

In [81]:
print('pollutant median',afterlock2020['PM2.5'].median())
print('pollutant mean',afterlock2020['PM2.5'].mean())
print('pollutant min',afterlock2020['PM2.5'].min())
print('pollutant max',afterlock2020['PM2.5'].max())
print(afterlock2020[['From Date','PM2.5']].loc[afterlock2020['PM2.5']==afterlock2020['PM2.5'].min()])
print(afterlock2020[['From Date','PM2.5']].loc[afterlock2020['PM2.5']==afterlock2020['PM2.5'].max()])

pollutant median 109.66
pollutant mean 127.50562091503271
pollutant min 7.25
pollutant max 528.92
    From Date  PM2.5
12 2020-08-13   7.25
     From Date   PM2.5
100 2020-11-09  528.92


PM2.5 concentration reached its lowest ever on 22 july 2020 . This was the lowest recored PM2.5 value in past 8 years in this given range of time.

# PM10

In [57]:
print('previous years statistical data')
print('pollutant median',noLock_data['PM1'].median())
print('pollutant mean',noLock_data['PM1'].mean())
print('pollutant min',noLock_data['PM1'].min())
print('pollutant max',noLock_data['PM1'].max())
print(noLock_data[['From Date','PM1']].loc[noLock_data['PM1']==noLock_data['PM1'].min()])
print(noLock_data[['From Date','PM1']].loc[noLock_data['PM1']==noLock_data['PM1'].max()])

pollutant median 200.045
pollutant mean 212.7714065934062
pollutant min 34.62
pollutant max 763.48
     From Date    PM1
645 2017-07-27  34.62
775 2018-07-27  34.62
     From Date     PM1
65  2013-05-28  763.48
195 2014-05-28  763.48
325 2015-05-28  763.48


In [60]:
print('statistics during lockdown')
print('pollutant median',lock2020['PM1'].median())
print('pollutant mean',lock2020['PM1'].mean())
print('pollutant min',lock2020['PM1'].min())
print('pollutant max',lock2020['PM1'].max())
print(lock2020[['From Date','PM1']].loc[lock2020['PM1']==lock2020['PM1'].min()])
print(lock2020[['From Date','PM1']].loc[lock2020['PM1']==lock2020['PM1'].max()])

statistics during lockdown
pollutant median 82.47
pollutant mean 88.98746153846155
pollutant min 29.67
pollutant max 260.5
     From Date    PM1
105 2020-07-07  29.67
    From Date    PM1
96 2020-06-28  260.5


In [82]:
print('pollutant median',afterlock2020['PM1'].median())
print('pollutant mean',afterlock2020['PM1'].mean())
print('pollutant min',afterlock2020['PM1'].min())
print('pollutant max',afterlock2020['PM1'].max())
print(afterlock2020[['From Date','PM1']].loc[afterlock2020['PM1']==afterlock2020['PM1'].min()])
print(afterlock2020[['From Date','PM1']].loc[afterlock2020['PM1']==afterlock2020['PM1'].max()])

pollutant median 184.46
pollutant mean 200.56032679738558
pollutant min 20.88
pollutant max 698.85
    From Date    PM1
12 2020-08-13  20.88
     From Date     PM1
100 2020-11-09  698.85


# NO

In [61]:
print('previous years statistical data')
print('pollutant median',noLock_data['NO'].median())
print('pollutant mean',noLock_data['NO'].mean())
print('pollutant min',noLock_data['NO'].min())
print('pollutant max',noLock_data['NO'].max())
print(noLock_data[['From Date','NO']].loc[noLock_data['NO']==noLock_data['NO'].min()])
print(noLock_data[['From Date','NO']].loc[noLock_data['NO']==noLock_data['NO'].max()])

previous years statistical data
pollutant median 10.754999999999999
pollutant mean 26.692868131868117
pollutant min 0.45
pollutant max 285.66
     From Date    NO
567 2017-05-10  0.45
     From Date      NO
424 2016-04-27  285.66


In [62]:
print('statistics during lockdown')
print('pollutant median',lock2020['NO'].median())
print('pollutant mean',lock2020['NO'].mean())
print('pollutant min',lock2020['NO'].min())
print('pollutant max',lock2020['NO'].max())
print(lock2020[['From Date','NO']].loc[lock2020['NO']==lock2020['NO'].min()])
print(lock2020[['From Date','NO']].loc[lock2020['NO']==lock2020['NO'].max()])

statistics during lockdown
pollutant median 2.935
pollutant mean 4.597653846153848
pollutant min 1.4
pollutant max 48.62
    From Date   NO
46 2020-05-09  1.4
     From Date     NO
121 2020-07-23  48.62


In [83]:
print('pollutant median',afterlock2020['NO'].median())
print('pollutant mean',afterlock2020['NO'].mean())
print('pollutant min',afterlock2020['NO'].min())
print('pollutant max',afterlock2020['NO'].max())
print(afterlock2020[['From Date','NO']].loc[afterlock2020['NO']==afterlock2020['NO'].min()])
print(afterlock2020[['From Date','NO']].loc[afterlock2020['NO']==afterlock2020['NO'].max()])

pollutant median 30.7
pollutant mean 50.35477124183009
pollutant min 1.92
pollutant max 277.32
    From Date    NO
35 2020-09-05  1.92
    From Date      NO
97 2020-11-06  277.32


# NO2

In [84]:
print('previous years statistical data')
print('pollutant median',noLock_data['NO2'].median())
print('pollutant mean',noLock_data['NO2'].mean())
print('pollutant min',noLock_data['NO2'].min())
print('pollutant max',noLock_data['NO2'].max())
print(noLock_data[['From Date','NO2']].loc[noLock_data['NO2']==noLock_data['NO2'].min()])
print(noLock_data[['From Date','NO2']].loc[noLock_data['NO2']==noLock_data['NO2'].max()])

previous years statistical data
pollutant median 62.135
pollutant mean 64.87863186813192
pollutant min 3.88
pollutant max 197.78
     From Date   NO2
643 2017-07-25  3.88
773 2018-07-25  3.88
     From Date     NO2
42  2013-05-05  197.78
172 2014-05-05  197.78
302 2015-05-05  197.78


In [85]:
print('statistics during lockdown')
print('pollutant median',lock2020['NO2'].median())
print('pollutant mean',lock2020['NO2'].mean())
print('pollutant min',lock2020['NO2'].min())
print('pollutant max',lock2020['NO2'].max())
print(lock2020[['From Date','NO2']].loc[lock2020['NO2']==lock2020['NO2'].min()])
print(lock2020[['From Date','NO2']].loc[lock2020['NO2']==lock2020['NO2'].max()])

statistics during lockdown
pollutant median 26.134999999999998
pollutant mean 27.38746153846153
pollutant min 7.05
pollutant max 61.67
     From Date   NO2
102 2020-07-04  7.05
    From Date    NO2
59 2020-05-22  61.67


In [87]:
print('pollutant median',afterlockprev['NO2'].median())
print('pollutant mean',afterlockprev['NO2'].mean())
print('pollutant min',afterlockprev['NO2'].min())
print('pollutant max',afterlockprev['NO2'].max())
print(afterlockprev[['From Date','NO2']].loc[afterlockprev['NO2']==afterlockprev['NO2'].min()])
print(afterlockprev[['From Date','NO2']].loc[afterlockprev['NO2']==afterlockprev['NO2'].max()])

pollutant median 67.55
pollutant mean 68.33136788048563
pollutant min 0.27
pollutant max 213.78
     From Date   NO2
980 2019-10-02  0.27
     From Date     NO2
715 2017-11-12  213.78


In [86]:
print('pollutant median',afterlock2020['NO2'].median())
print('pollutant mean',afterlock2020['NO2'].mean())
print('pollutant min',afterlock2020['NO2'].min())
print('pollutant max',afterlock2020['NO2'].max())
print(afterlock2020[['From Date','NO2']].loc[afterlock2020['NO2']==afterlock2020['NO2'].min()])
print(afterlock2020[['From Date','NO2']].loc[afterlock2020['NO2']==afterlock2020['NO2'].max()])

pollutant median 44.06
pollutant mean 47.65071895424835
pollutant min 2.48
pollutant max 186.44
    From Date   NO2
16 2020-08-17  2.48
    From Date     NO2
97 2020-11-06  186.44
